In [1]:
import torch
import gpytorch
from matplotlib import pyplot as plt

torch.manual_seed(0)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Generate synthetic regression data
train_x = torch.linspace(0, 1, 100)
train_y = torch.sin(train_x * (2 * torch.pi)) + 0.2 * torch.randn(train_x.size())

# Add batch dimension
train_x = train_x.unsqueeze(-1)

In [26]:
# ---- Define the GP Model ----
class RFFGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_rff=100):
        super().__init__(train_x, train_y, likelihood)

        # Mean
        self.mean_module = gpytorch.means.ConstantMean()

        # Base kernel
        base_kernel = gpytorch.kernels.RBFKernel(ard_num_dims=train_x.shape[-1])

        # RFF approximation wrapped in ScaleKernel
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RFFKernel(num_rff, base_kernel=base_kernel)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Define likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = RFFGPModel(train_x, train_y, likelihood, num_rff=256)

# Print model information
print(model)
for name, param in model.named_parameters():
    print(name, param.data)

RFFGPModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): RFFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
)
likelihood.noise_covar.raw_noise tensor([0.])
mean_module.raw_constant tensor(0.)
covar_module.raw_outputscale tensor(0.)
covar_module.base_kernel.raw_lengthscale tensor([[0.]])


In [27]:
model.state_dict()
# model.covar_module.base_kernel.num_samples

OrderedDict([('likelihood.noise_covar.raw_noise', tensor([0.])),
             ('likelihood.noise_covar.raw_noise_constraint.lower_bound',
              tensor(1.0000e-04)),
             ('likelihood.noise_covar.raw_noise_constraint.upper_bound',
              tensor(inf)),
             ('mean_module.raw_constant', tensor(0.)),
             ('covar_module.raw_outputscale', tensor(0.)),
             ('covar_module.base_kernel.raw_lengthscale', tensor([[0.]])),
             ('covar_module.base_kernel.raw_lengthscale_constraint.lower_bound',
              tensor(0.)),
             ('covar_module.base_kernel.raw_lengthscale_constraint.upper_bound',
              tensor(inf)),
             ('covar_module.raw_outputscale_constraint.lower_bound',
              tensor(0.)),
             ('covar_module.raw_outputscale_constraint.upper_bound',
              tensor(inf))])

In [23]:
# Hyperparameter optimization
model.train()
likelihood.train()

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iter = 200
for i in range(training_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    if i % 50 == 0:
        print(f"Iter {i:3d}/{training_iter} - Loss: {loss.item():.3f}")
    optimizer.step()

RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:

# 4. Optimize hyperparameters



# 5. Switch to eval mode and make predictions
model.eval()
likelihood.eval()

test_x = torch.linspace(0, 1, 200).unsqueeze(-1)
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    pred = likelihood(model(test_x))

# Extract mean and confidence interval
mean = pred.mean
lower, upper = pred.confidence_region()

# 6. Plot results
plt.figure(figsize=(8, 4))
plt.plot(train_x.numpy(), train_y.numpy(), "k*", label="Training data")
plt.plot(test_x.numpy(), mean.numpy(), "b", label="Predictive mean")
plt.fill_between(
    test_x.squeeze().numpy(),
    lower.numpy(),
    upper.numpy(),
    alpha=0.3,
    label="Confidence",
)
plt.legend()
plt.show()
